# Main Extraction Engine
Run as many times as needed to get full list of players. 

In [1]:
# import python libraries and nba library
import pandas as np
import nba_api as nba
from matplotlib import pyplot as plt
import pandas as pd
import socket
import logging
import os
import glob

In [2]:
# Get all files in working data directory
path = os.getcwd()
path = path.replace("notebooks\\basketballProject\\extraction","data\\basketballProject")+'\\'+'extraction-files\\'
csv_files = glob.glob(os.path.join(path,"*.csv"))
write_path = "C:/Github Repo/NBA-Analytics/data/basketballProject/extraction-files/nba_data_"
# write_path = "../../data/basketballProject/extraction-files/nba_data_"

In [3]:
# Check what players you have already analyzed to avoid duplication
check_df = pd.DataFrame()
for file in csv_files:

    df = pd.read_csv(file)
    frames = [check_df, df]
    check_df = pd.concat(frames)

In [4]:
try:
    unique_ids = check_df.PLAYER_ID.unique()
except:
    pass

In [5]:
display(check_df)

,Unnamed: 0,PLAYER_ID,full_name,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,0,203457,Nerlens Noel,2014-15,0,1610612755,PHI,21.0,75,71,...,0.609,185,426,611,128,133,142,146,208,744
1,1,203457,Nerlens Noel,2015-16,0,1610612755,PHI,22.0,67,62,...,0.590,154,388,542,122,118,100,158,192,747
2,2,203457,Nerlens Noel,2016-17,0,1610612755,PHI,23.0,29,7,...,0.683,46,98,144,30,43,26,29,71,257
3,3,203457,Nerlens Noel,2016-17,0,1610612742,DAL,23.0,22,12,...,0.708,47,103,150,20,22,24,22,55,188
4,4,203457,Nerlens Noel,2016-17,0,0,TOT,23.0,51,19,...,0.694,93,201,294,50,65,50,51,126,445
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2400,0,1630227,Daishen Nix,2021-22,0,1610612745,HOU,20.0,24,0,...,0.533,7,26,33,40,15,0,26,21,77
2401,1,1630227,Daishen Nix,2022-23,0,1610612745,HOU,20.0,28,0,...,0.789,10,40,50,76,23,3,40,27,103
2402,0,1630192,Zeke Nnaji,2020-21,0,1610612743,DEN,20.0,42,1,...,0.800,14,51,65,10,8,4,7,29,136
2403,1,1630192,Zeke Nnaji,2021-22,0,1610612743,DEN,21.0,41,1,...,0.631,56,91,147,17,16,13,23,79,270


In [151]:
# import teams, players, and sort them into active or all players
from nba_api.stats.static import teams, players
# nba_players = players.get_players()
active_players = players.get_active_players()
teams = teams.get_teams()

In [152]:
# import use-case specific functions
from nba_api.stats.endpoints import playercareerstats

In [153]:
# Add entry for each player on whether they have been analyzed or not. 
# Default is not analyzed
for player in active_players:
    player['Parsed ?'] = False
    try:
        if player["id"] in unique_ids:
            player["Parsed ?"] = True
    except:
        pass

In [154]:
# Initialize collection dataframe
master_list = pd.DataFrame()

# Create temp dataframe and initialize count to serve as a staging 
# Dataframe and break up the merging into small batches
temp_df = pd.DataFrame()
count = 0

In [155]:
# Parse thru each active player
first_id = 0
last_id = 0
try:
    for player in active_players:

        # Check if they have been already analyzed
        if player['Parsed ?'] == True:
            pass
        
        # If they haven't, proceed
        else:
            passedId = player['id']
            player_name = player["full_name"]

            # Get player stats based on their ID
            career_stats = playercareerstats.PlayerCareerStats(player_id = passedId).get_data_frames()[0]
            
            # Get ID of first and last players analyzed for file naming purposes
            if first_id == 0:
                first_id = passedId
            last_id = passedId

            # Add player names to resulting dataframe to keep track of players
            career_stats.insert(1,"full_name",player_name)

            # Append resulting dataframe to temp dataframe
            frames = [temp_df,career_stats]
            temp_df = pd.concat(frames)

            # Set analyzed flag to true
            player['Parsed ?'] = True
            count = count + 1
            # Once enough volume is reached in the temp dataframe, append it to the main and re-initialize the dataframe
            if (count%20) == 0:
                frames = [master_list, temp_df]
                master_list = pd.concat(frames)
                temp_df = pd.DataFrame()
                print("Temp Full - Resetting - "+str(count))
except:
    frames = [master_list, temp_df]
    master_list = pd.concat(frames)
    temp_df = pd.DataFrame()
    print("Temp Full - Resetting - "+str(count))
    print("TimeoutError")

LiAngelo Ball
Jules Bernard
Marcus Bingham Jr.
Jamaree Bouyea
Gabe Brown
Jalen Crutcher
Darius Days
Anthony Duruji
Abdul Gaddy
Collin Gillespie
Grant Golden
RaiQuan Gray
Mouhamadou Gueye
Chet Holmgren
Matt Hurt
Quenton Jackson
Drake Jeffries
Johnny Juzang
Noah Kirkwood
Justin Lewis
Temp Full - Resetting - 20
Makur Maker
Isaiah Miller
Trhae Mitchell
Nerlens Noel
Jaylen Nowell
Frank Ntilikina
Kendrick Nunn
Jusuf Nurkic
David Nwaba
Jordan Nwora
Royce O'Neale
Chuma Okeke
Josh Okogie
Onyeka Okongwu
Isaac Okoro
KZ Okpala
Victor Oladipo
Kelly Olynyk
Eugene Omoruyi
Cedi Osman
Temp Full - Resetting - 40
Kelly Oubre Jr.
Eric Paschall
Chris Paul
Cameron Payne
Gary Payton II
Jamorko Pickett
Nate Pierre-Louis
Theo Pinson
Scotty Pippen Jr.
Daeqwon Plowden
Mason Plumlee
Jakob Poeltl
Aleksej Pokusevski
Jordan Poole
Kevin Porter Jr.
Michael Porter Jr.
Otto Porter Jr.
Bobby Portis
Kristaps Porzingis
Dwight Powell
Temp Full - Resetting - 60
Norman Powell
Jason Preston
Joshua Primo
Taurean Prince
Payton P

In [156]:
if first_id == 0:
    print("empty")
    pass
else:
    master_list.to_csv(write_path+str(first_id)+"to"+str(last_id)+".csv")